<a href="https://colab.research.google.com/github/ydesquitado/DSC1115/blob/main/DSC1115_DESQUITADO_FA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DSC1115 Predictive Analytics - Formative Assessment 2**
## **2021025081 DESQUITADO, Yuichi P.**

### Task 2 - Image Classification with CNNs
### Dataset: CIFAR-10Links to an external site. (or use a smaller subset from Kaggle or TensorFlow Datasets).
### **Goal: *Classify images into 10 categories.***

**Steps:**
* Load and preprocess dataset (resize, normalize).
* Build a CNN with 2-3 convolutional layers, pooling, and dropout.
* Train the model and compare performance with and without transfer learning (e.g., using MobileNet or ResNet).
* Report accuracy and confusion matrix.

**Deliverables:**
* Python notebooks (.ipynb) with code and markdown explanations.
* Brief report comparing MLP vs. CNN models, and the effect of transfer learning.

# **Without Transfer Learning**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

(X_train, y_train), (X_test, y_test) = cifar10.load_data()            # Loads the dataset into the program

X_train, X_test = X_train / 255.0, X_test / 255.0                     # Normalizes the data

y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)    # Converts the target values to categorical
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [ ]:
modelCNN = Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape = (32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation = 'relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation = 'relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(10, activation = 'softmax'),
])

modelCNN.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

modelCNN.fit(X_train, y_train, epochs = 10, batch_size = 64, validation_data = (X_test, y_test))

In [ ]:
modelMLP = Sequential([
    Flatten(input_shape = (32, 32, 3)),
    Dense(512, activation = 'relu'),
    Dropout(0.5),
    Dense(256, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

modelMLP.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

modelMLP.fit(X_train, y_train, epochs = 10, batch_size = 64, validation_data = (X_test, y_test))

In [ ]:
print("CNN Accuracy:", modelCNN.evaluate(X_test, y_test)[1])
print("MLP Accuracy:", modelMLP.evaluate(X_test, y_test)[1])

In [ ]:
y_pred_cnn = np.argmax(modelCNN.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_cnn)

plt.figure(figsize = (10, 8))
sns.heatmap(conf_matrix,
            annot = True, fmt = "d", cmap = "Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - CNN")
plt.show()

In [ ]:
y_pred_mlp = np.argmax(modelMLP.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_mlp)

plt.figure(figsize = (10, 8))
sns.heatmap(conf_matrix,
            annot = True, fmt = "d", cmap = "Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - MLP")
plt.show()

# **With Transfer Learning**

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

modelBase = VGG16(weights = 'imagenet',
                  include_top = False,
                  input_shape = (32, 32, 3))      # The base model used for Transfer Learning

for layer in modelBase.layers:
    layer.trainable = False

modelCNNTL = Sequential([
    modelBase,
    Flatten(),
    Dense(256, activation = 'relu'),
    Dropout(0.5),
    Dense(10, activation = 'softmax')
])

modelCNNTL.compile(optimizer = Adam(learning_rate = 0.0001),
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])
modelCNNTL.fit(X_train, y_train, epochs = 10, batch_size = 64, validation_data = (X_test, y_test))

In [ ]:
print("CNN with Transfer Learning Accuracy:", modelCNNTL.evaluate(X_test, y_test)[1])

In [ ]:
y_pred_cnn_tl = np.argmax(modelCNNTL.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_cnn_tl)

plt.figure(figsize = (10, 8))
sns.heatmap(conf_matrix,
            annot = True, fmt = "d", cmap = "Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - CNN with Transfer Learning")
plt.show()

In [ ]:
predicted = {
    'True' : y_true,
    'CNN' : y_pred_cnn,
    'MLP' : y_pred_mlp,
    'CNN TL' : y_pred_cnn_tl
}

predictedDF = pd.DataFrame(predicted)
predictedDF.head(20)